In [1]:
import pandas as pd
import optuna

import sklearn.linear_model as sklm
import sklearn.model_selection as skms
import sklearn.metrics as metrics
import sklearn.pipeline as skpl
import sklearn.preprocessing as skpp
import sklearn.compose as skcmp
import category_encoders as ce

import lightgbm as lgb

from preprocessing import *
from ucimlrepo import fetch_ucirepo

import gc

In [2]:
train_df = pd.read_csv("data/train.csv", index_col="id")
test_df = pd.read_csv("data/test.csv", index_col="id")
orig_df = fetch_ucirepo(id=848)['data']['original']

CONT_FEATS = [
    "cap-diameter",
    "stem-height",
    "stem-width"
]
CAT_FEATS = [c for c in train_df.columns if c not in CONT_FEATS]
RESPONSE_COL = "class"

train_df = convert_cols(train_df, CONT_FEATS, CAT_FEATS)
test_df = convert_cols(test_df, CONT_FEATS, CAT_FEATS)
orig_df = convert_cols(orig_df, CONT_FEATS, CAT_FEATS)

train_df = null_all_non_original_categories(train_df, orig_df, CAT_FEATS)
test_df = null_all_non_original_categories(test_df, orig_df, CAT_FEATS)

d:\Programming\Workspaces\Python3 Workspace\Misc Data Analysis\Kaggle Playground Competition\S4E8\preprocessing.py:17: UserWarning: 'class' not found in dataframe
  warnings.warn(f"'{c}' not found in dataframe")
d:\Programming\Workspaces\Python3 Workspace\Misc Data Analysis\Kaggle Playground Competition\S4E8\preprocessing.py:31: UserWarning: 'class' not found in dataframe
  warnings.warn(f"'{c}' not found in dataframe")


In [3]:
for df in [train_df, orig_df]:
    df["class"] = df["class"].cat.rename_categories({"e":0, "p":1})

In [6]:
x_train, x_val, y_train, y_val = skms.train_test_split(train_df.drop(columns=[RESPONSE_COL]), train_df[RESPONSE_COL], stratify = train_df[RESPONSE_COL])


def objective(trial):
    model = lgb.LGBMClassifier(
        boosting_type="dart",
        n_jobs=-1,

        num_leaves=trial.suggest_int("num_leaves", 15, 45),
        max_depth=trial.suggest_int("max_depth", -1, 50),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1),
        n_estimators=trial.suggest_int("n_estimators", 1, 500),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.05, 1),
        reg_lambda=trial.suggest_int("reg_lambda", 0, 20),
        reg_alpha=trial.suggest_int("reg_alpha", 0, 20),
    )

    model.fit(x_train, y_train, eval_metric="mcc")

    preds = model.predict(x_val)

    return metrics.matthews_corrcoef(y_val, preds)


In [7]:
# optuna.delete_study(study_name="dart_tuning", storage="sqlite:///optuna.sqlite3")

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///optuna.sqlite3",
    study_name="dart_tuning",
    load_if_exists=True
)
study.optimize(objective, n_trials=50)
gc.collect()

[I 2024-08-08 14:37:14,504] Using an existing study with name 'dart_tuning' instead of creating a new one.


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.066093 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 898
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


[I 2024-08-08 14:38:50,343] Trial 5 finished with value: 0.9754634649449754 and parameters: {'num_leaves': 39, 'max_depth': 25, 'learning_rate': 0.05163482386743505, 'n_estimators': 209, 'colsample_bytree': 0.8930219666873452, 'reg_lambda': 10, 'reg_alpha': 14}. Best is trial 5 with value: 0.9754634649449754.


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.067631 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 898
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


[I 2024-08-08 14:39:56,093] Trial 6 finished with value: 0.8770478526595541 and parameters: {'num_leaves': 15, 'max_depth': 36, 'learning_rate': 0.017944482876098282, 'n_estimators': 204, 'colsample_bytree': 0.3417536931845077, 'reg_lambda': 14, 'reg_alpha': 7}. Best is trial 5 with value: 0.9754634649449754.


...

[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.076960 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 898
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


[I 2024-08-08 18:03:56,572] Trial 53 finished with value: 0.9810241220490888 and parameters: {'num_leaves': 43, 'max_depth': 47, 'learning_rate': 0.06422674141877682, 'n_estimators': 371, 'colsample_bytree': 0.5806730374571512, 'reg_lambda': 18, 'reg_alpha': 13}. Best is trial 50 with value: 0.983488929774915.


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.072177 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 898
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


[I 2024-08-08 18:10:33,468] Trial 54 finished with value: 0.9831456792835306 and parameters: {'num_leaves': 44, 'max_depth': 48, 'learning_rate': 0.08439389512528117, 'n_estimators': 479, 'colsample_bytree': 0.47844417766558034, 'reg_lambda': 18, 'reg_alpha': 11}. Best is trial 50 with value: 0.983488929774915.


3599

In [8]:
best_model = lgb.LGBMClassifier(
    boosting_type="dart",
    n_jobs=-1,

    **study.best_params
)
best_model.fit(train_df.drop(columns=["class"]), train_df["class"])
preds = best_model.predict(test_df)
out_df = pd.DataFrame({"class":preds}, index=test_df.index)

[LightGBM] [Info] Number of positive: 1705396, number of negative: 1411549
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.098972 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 898
[LightGBM] [Info] Number of data points in the train set: 3116945, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


In [9]:
out_df["class"] = out_df["class"].replace({0:"e", 1:"p"})
out_df.to_csv("tuned_dart_100_v1.csv")